In [1]:
from cosapp.base import System, Port, BaseConnector
import numpy as np
from cosapp.drivers import RungeKutta
import plotly.graph_objs as go
from cosapp.recorders import DataFrameRecorder
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
def RotMat3D(ang):
    
    #ang[0] = phi, ang[1] = theta, ang[2] = psi
    
    l1 = np.array([np.cos(ang[2])*np.cos(ang[1]), -np.sin(ang[2])*np.cos(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                    np.sin(ang[2])*np.sin(ang[0]) + np.cos(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l2 = np.array([np.sin(ang[2])*np.cos(ang[1]),  np.cos(ang[2])*np.cos(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.sin(ang[0]),
                   -np.cos(ang[2])*np.sin(ang[0]) + np.sin(ang[2])*np.sin(ang[1])*np.cos(ang[0])])
    l3 = np.array([-np.sin(ang[1]), np.cos(ang[1])*np.sin(ang[0]), np.cos(ang[1])*np.cos(ang[0])])
    
    A = np.round(np.array([l1,l2,l3]), 5)
    
    return A

In [3]:
class PosPort(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Velocity Components", unit = 'm')
        self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            sink.val = np.matmul(RotMat3D(-source.ang), source.val)

In [4]:
class VelPort(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Velocity Components", unit = 'm/s')
        self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            sink.val = np.matmul(RotMat3D(-source.ang), source.val)

In [5]:
class AclPort(Port):
    """Position Port """
    def setup(self):
        self.add_variable('val', np.zeros(3), desc = "Acceleration Components", unit = 'm/s**2')
        self.add_variable('ang', np.zeros(3), desc = "Axes' Euler's Angles", unit = '')
        

    class Connector(BaseConnector):
        """Custom connector for `PosPort` objects
        """
        def __init__(self, name: str, sink: Port, source: Port, *args, **kwargs):
            super().__init__(name, sink, source)

        def transfer(self) -> None:
            sink = self.sink
            source = self.source
            
            sink.val = np.matmul(RotMat3D(-source.ang), source.val)

In [6]:
class Earth(System):
    
    def setup(self):
        
        #Parent-Child connections
        self.add_child(Gravity('Grav'))
        self.add_child(Atmosphere('Atmo'), pulling = ['V_wind'])
        self.add_child(Rocket('Rocket'))
        self.add_child(Trajectory('Traj'))
        
        #Child-Child connections
        self.connect(self.Rocket, self.Traj, ['v_out'])
        self.connect(self.Rocket, self.Grav, ['g', 'ar'])
        self.connect(self.Traj, self.Atmo, ['r'])
        
        #Execution Order
        self.exec_order = ['Grav', 'Atmo', 'Rocket', 'Traj']
        

In [7]:
class Rocket(System):
    
    def setup(self):
        
        #Parent-Child connections
        self.add_child(Dynamics('Dyn'), pulling = ['a', 'aa', 'g'])
        self.add_child(Thrust('Thrust'))
        self.add_child(Geometry('Geom'))
        self.add_child(Aerodynamics('Aero'))
        
        #Child-Child connections
        self.connect(self.Dyn, self.Thrust, ['Fp', 'Mp'])
        
        #Transient Variables
        self.add_transient('v', der = 'a', desc = "Rocket Velocity")
        self.add_transient('av', der = 'aa', desc = "Rocket Angular Velocity")
        self.add_transient('ar', der = 'av', desc = "Rocket Angular Position")
        
        #Rocket outputs
        self.add_output(VelPort, 'v_out')
        
        #Execution Order
        self.exec_order = ['Geom', 'Aero', 'Thrust', 'Dyn']
        
    def compute(self):
        
        self.v_out.val = self.v
        self.v_out.ang = self.ar
    

In [8]:
class Gravity(System):
    
    def setup(self):
        
        #System Constants
        self.add_inward('G', 6.6743*10**(-11), desc = "Gravitational Constant")
        self.add_inward('M', 5.972*10**24, desc = "Earth's Mass")
        self.add_inward('R', 6.371*10**6, desc = "Earth's Radius")
        
        #Rocket inputs
        self.add_inward('ar', np.zeros(3), desc = "Rocket's Angular Position")
        
        #Trajectory inputs
        self.add_inward('r', np.zeros(3), desc = "Rocket's Position")
        
        #Gravity outputs
        self.add_output(AclPort, 'g')
        
    def compute(self):
        
        self.g.val = np.array([0., 0., -self.G*self.M/(self.R + self.r[-1])**2])
        self.g.ang = -self.ar
        
        

In [9]:
class Atmosphere(System):
    
    def setup(self):
        
        #Parent-Child connections
        self.add_child(Density('Dens'), pulling = ['r'])
        self.add_child(Pressure('Pres'), pulling = ['r'])
        self.add_child(Wind('Wind'), pulling = ['V_wind'])
        
        #Execution order
        self.exec_order = ['Dens', 'Pres', 'Wind']
        

In [10]:
class Density(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('rho0', 1.225, desc = "Air Density at Sea Level")
        
        #Trajectory inputs
        self.add_inward('r', np.zeros(3), desc = "Rocket's Position")
        
        #Density outputs
        self.add_outward('rho', 1.225, desc = "Air Density at Rocket's Height")
        
    def compute(self):
        
        self.rho = self.rho0 * (20000. - self.r[-1]) / (20000. + self.r[-1])
        
        

In [11]:
class Pressure(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('P0', 101325., desc = "Atmospheric Pressure at Sea Level")
        
        #Trajectory inputs
        self.add_inward('r', np.zeros(3), desc = "Rocket's Position")
        
        #Pressure outputs
        self.add_outward('P', 101325., desc = "Atmospheric Pressure at Rocket's Height")
        
    def compute(self):
        
        self.P = self.P0 * (20000. - self.r[-1]) / (20000. + self.r[-1])
        

In [12]:
class Wind(System):
    
    def setup(self):
        
        #Wind inputs
        self.add_inward('V', 5., desc = "Wind speed")
        self.add_inward('eps', np.pi/6, desc = "Wind Direction")
        
        #Wind outputs
        self.add_output(VelPort, 'V_wind')
        
    def compute(self):
        
        self.V_wind.val = np.array([self.V, 0., 0.])
        self.V_wind.ang = np.array([0., 0., -self.eps])
        

In [13]:
class Trajectory(System):
    
    def setup(self):
        
        self.add_input(VelPort, 'v_out')
        
        self.add_transient('r', der = 'v_out.val', desc = 'Rocket Position')
        

In [14]:
class Dynamics(System):
    
    def setup(self):
        
        #Rocket inputs
        self.add_inward('v', np.zeros(3), desc = "Rocket's Velocity", unit = 'm/s')
        self.add_inward('av', np.zeros(3), desc = "Rocket's Angular Velocity", unit = '1/s')
        
        #Thrust inputs
        self.add_inward('Fp', np.zeros(3), desc = "Thrust Force", unit = 'N')
        self.add_inward('Mp', np.zeros(3), desc = "Thrust Moment", unit = 'N*m')
        
        #Geometry inputs
        self.add_inward('m', 100., desc = "Rocket Mass", unit = 'kg')
        self.add_inward('I', np.array([10., 100., 100.]), desc = "Rocket Moments of Inertia")
        
        #Gravity inputs
        self.add_input(AclPort, 'g')
        
        #Dynamics outputs
        self.add_outward('a', np.zeros(3), desc = "Rocket Acceleration", unit = 'm/s**2')
        self.add_outward('aa', np.zeros(3), desc = "Rocket Angular Acceleration", unit = '1/s**2')
        
    def compute(self):
        
        self.a[0] = (self.Fp[0])/self.m + self.g.val[0] + self.av[2]*self.v[1] - self.av[1]*self.v[2]
        self.a[1] = (self.Fp[1])/self.m + self.g.val[1] + self.av[0]*self.v[2] - self.av[2]*self.v[0]
        self.a[2] = (self.Fp[2])/self.m + self.g.val[2] + self.av[1]*self.v[0] - self.av[0]*self.v[1]
        
        self.aa[0] = (self.Mp[0] + (self.I[1] - self.I[2])*self.av[1]*self.av[2])/self.I[0]
        self.aa[1] = (self.Mp[1] + (self.I[2] - self.I[0])*self.av[2]*self.av[0])/self.I[1]
        self.aa[2] = (self.Mp[2] + (self.I[0] - self.I[1])*self.av[0]*self.av[1])/self.I[2]
        
        

In [15]:
class Thrust(System):
    
    def setup(self):
        
        #System constants
        self.add_inward('g0', 9.81, desc = "Gravity on Earth's Surface", unit = 'm/s**2')
        
        #Rocket inputs
        self.add_inward('qp', 10., desc = "Engine's Propulsive Debit", unit = 'kg/s')
        self.add_inward('isp', 100., desc = "Specific Impulsion in Vacuum", unit = 's')
        self.add_inward('gc', 1., desc = "GC Distance")
        
        #Thrust outputs
        self.add_outward('Fp', np.zeros(3), desc = "Thrust Force", unit = 'N')
        self.add_outward('Mp', np.zeros(3), desc = "Thrust Moment", unit = 'N*m')
        
    def compute(self):
        
        self.Fp[0] = self.qp*self.g0*self.isp
        self.Fp[1] = 0.
        self.Fp[2] = 0.
        
        self.Mp[0] = 0.
        self.Mp[1] = 0.
        self.Mp[2] = 0.
        
        

In [16]:
class Geometry(System):
    
    def setup(self):
        
        self.add_inward('a', 1.)
        

In [17]:
class Aerodynamics(System):
    
    def setup(self):
        
        self.add_inward('a', 1.)
        

In [18]:
#Main function

#Time-step
dt = 0.01

#Add RungeKutta driver
earth = Earth('earth')
driver = earth.add_driver(RungeKutta(order=4, dt=dt))
driver.time_interval = (0, 9)

# Add a recorder to capture time evolution in a dataframe
driver.add_recorder(
    DataFrameRecorder(includes=['Traj.r', 'Rocket.v', 'Rocket.a']),
    period=1,
)

#Initial conditions and constants
driver.set_scenario(
    init = {
        'Rocket.v' : np.zeros(3),
        'Rocket.ar' : np.array([0., np.pi/2, 0.]),
        'Rocket.av' : 0.*np.array([np.pi/20, np.pi/20, 0.])
    })


earth.run_drivers()

In [64]:
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)
time = np.asarray(data['time'])
traj = np.asarray(data['Traj.r'].tolist())
velo = np.asarray(data['Rocket.v'].tolist())
acel = np.asarray(data['Rocket.a'].tolist())

In [65]:
traj

array([[   0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,   44.14001329],
       [   0.        ,    0.        ,  176.56005315],
       [   0.        ,    0.        ,  397.26011958],
       [   0.        ,    0.        ,  706.24021259],
       [   0.        ,    0.        , 1103.50033217],
       [   0.        ,    0.        , 1589.04047833],
       [   0.        ,    0.        , 2162.86065106],
       [   0.        ,    0.        , 2824.96085036],
       [   0.        ,    0.        , 3575.34107624]])

In [66]:
velo

array([[  0.        ,   0.        ,   0.        ],
       [ 88.28002657,   0.        ,   0.        ],
       [176.56005315,   0.        ,   0.        ],
       [264.84007972,   0.        ,   0.        ],
       [353.1201063 ,   0.        ,   0.        ],
       [441.40013287,   0.        ,   0.        ],
       [529.68015944,   0.        ,   0.        ],
       [617.96018602,   0.        ,   0.        ],
       [706.24021259,   0.        ,   0.        ],
       [794.52023916,   0.        ,   0.        ]])

In [67]:
acel

array([[88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ],
       [88.28002657,  0.        ,  0.        ]])

In [58]:
earth

earth - Earth